<a href="https://colab.research.google.com/github/matthewberry/uiuc_com_dsp/blob/master/DSP_personalized_medicine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using This Notebook

This notebook is an interactive environment that combines explanatory text with executable code. You will step through an example analysis, which can then serve as a starting point for a project of your own design.

If you are new to notebooks, you might find this introduction helpful: [Overview of Colaboratory Features](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) (Because this notebook uses R, not python, code examples from the introduction will not execute properly here.) You might also want to refer to the [R documentation](https://cran.r-project.org/doc/manuals/r-release/R-intro.html).

**Important Note**: After a period of inactivity (Google does not specify exactly how long), Google will disconnect your notebook from the virtual machine that had been running it. When you return, Google will connect to a new virtual machine. **If this happens and you find cells that previously ran without error have suddenly stopped working, re-run all of the notebook's cells again, starting at the top.**

### Before You Proceed

In order to run the notebook, you will need your own copy of it, along with your own copy of the data. Here are the steps to follow:

1. If you have not already enabled Google Apps @ Illinois, which allows you to use Google Drive, Google Docs, and so on with your illinois.edu account, [enable Google Apps @ Illinois](https://answers.uillinois.edu/illinois/page.php?id=55049).
2. Check the currently active Google account on this notebook. Look near the top-right corner of the screen for either a `Sign In` button or a round icon containing either a letter or your profile photo. If you see a `Sign In` button, click it and follow the prompts to sign in with your illinois.edu account. Otherwise, click the icon to open a popup that identifies the currently active Google account. If the account is not your illinois.edu account, switch to your illinois.edu account (you might have to click `Add Account` if it is not already an option in the list).
3. In the `File` menu above, select `Save a copy in Drive...`. This step will create a new browser tab containing your copy of the notebook. At this point, you can close the old browser tab that contained the original copy of the notebook.
4. Open the `File` menu and select `Locate in Drive`, which will show you where you can find the notebook if you need to open it later.

## Environment Setup

The cell below sets up the environment for running the analysis. Run the cell and wait for it to complete, which will only take a few seconds. You will see some informational messages, which you can ignore.

In [0]:
install.packages("randomForest", repos = "http://cran.rstudio.com/")

library(randomForest)
library(tidyverse)
library(rpart)

system("rm -rf /content/sample_data")

# Personalized Medicine Example: Sepsis Treatment

Personalized medicine draws a lot of attention in medical research. The goal of personalized medicine is to make a tailored decision for each patient, such that his/her clinical outcome can be optimized.

Let’s consider a simulated data set derived from the [SIDES package](http://biopharmnet.com/subgroup-analysis-software/). In this data set, 470 patients and 14 variables are observed. The variables are listed below.

*   `Health`: Health outcome (larger the better)
*   `THERAPY`: 1 for active treatment, 0 for the control treatment
*   `TIMFIRST`: Time from first sepsis-organ fail to start drug
*   `AGE`: Patient age in years
*   `BLLPLAT`: Baseline local platelets
*   `blSOFA`: Sum of baseline sofa score (cardiovascular, hematology, hepatorenal, and respiration scores)
*   `BLLCREAT`: Base creatinine
*   `ORGANNUM`: Number of baseline organ failures
*   `PRAPACHE`: Pre-infusion apache-ii score
*   `BLGCS`: Base GLASGOW coma scale score
*   `BLIL6`: Baseline serum IL-6 concentration
*   `BLADL`: Baseline activity of daily living score
*   `BLLBILI`: Baseline local bilirubin
*   `BEST`: The true best treatment suggested by doctors. **You should not use this variable when fitting models!**

For each patient, sepsis was observed during their hospital stay. Hence, one of the two treatments (indicated by variable `THERAPY`) must be chosen to prevent further adverse events. After the treatment, the patient's health outcome (`Health`) was measured, with a larger value being the better outcome.

Run the cell below to load the data set and display the first few rows. It will only take a few seconds to complete. You will see a warning message about a missing column name, which is explained in the comment and addressed by the code that follows the comment.

#### Working with Input and Output Files

You will see that the cell below reads the input csv file from a URL. For your independent project, you can use the same approach if you like, or you can use
Colab's built-in tools for uploading and downloading files: Find the file-folder icon in the narrow bar that runs along the left side of this screen. Click it to reveal the `Files` panel. Here, you can upload files that will be accessible to your notebook by using the path `/content/name-of-your-file`. Your notebook can also create and modify files within the `/content/` directory, which you can then download via the `Files` panel.

In [0]:
Sepsis <- read_csv("http://drive.google.com/uc?id=1v_Pq_DTmE0zlSzzVHg3f_5cx-_lSCxnX")
# the first, unnamed column in the csv file will be assigned the name X1
# (R will print a warning message about the missing column name)
# we want to use X1 not as an observed variable but as the index
Sepsis <- column_to_rownames(Sepsis, var = "X1")
head(Sepsis)

# Virtual Twins: Introduction and Tuning

We would like to use the data set to predict the best treatment for a new patient. A strategy called [Virtual Twins](https://onlinelibrary.wiley.com/doi/abs/10.1002/sim.4322) was proposed by Foster et al. (2011) to tackle this problem. Here we consider a simpler version of the method.

We will fit two random forests to model the outcome `Health`: one model is based on all patients who received treatment 1, and the other model is based on all patients who received treatment 0. Denote these two models as $\widehat f_1(x)$ and $\widehat f_0(x)$, respectively. When a new patient arrives, we will use the two models to calculate the expected health status for the new patient, and we will suggest the treatment associated with the model that gives a larger prediction value. In other words, for a new $x^{*}$, we compare $\widehat f_1(x^{*})$ and $\widehat f_0(x^{*})$ and suggest the treatment whose model produced the higher value.

We will build the models with the random forest algorithm. Our first step is to tune the `mtry` and `nodesize` parameters for the data set.

Read the code and comments in the cell below. Run the cell, which will take about seven minutes to complete.

In [0]:
# n_simulations is the number of random simulations we will run
n_simulations <- 100

# n_patients is the number of patients
n_patients <- nrow(Sepsis)

# all_tune is a dataframe with one row for each combination of nodesize and mtry
# that we want to evaluate
# you can find guidance on values for these and other random forest parameters
# in machine learning literature
all_tune <- expand.grid(
            "nodesize" = c(1, 5, 10),
            "mtry" = c(1, 4, 12))

# error_vals is a matrix of the error values from each simulation 
error_vals <- matrix(NA, nrow(all_tune), n_simulations)

# initialize the random seed before starting the simulations for reproducibility
set.seed(1)

# for each simulation...
for (k in 1:n_simulations) {
  # create a training set consisting of 75% of the patients and a testing set
  # consisting of the remaining 25% of the patients
  intrain <- sample(1:n_patients, 0.75*n_patients)

  train <- Sepsis[intrain, ]
  test <- Sepsis[-intrain, ]

  # for each combination of nodesize and mtry...
  for (j in 1:nrow(all_tune)) {
    # build the model for treatment 0 and the model for treatment 1
    # note the following for model0 and model1:
    # - BEST is not used in building the model
    # - the models are built only from the patients in the training set (not the
    #   testing test) who received the therapy associated with the model
    # - mtry and nodesize come from all_train
    model0 <- randomForest(Health ~ . - BEST, data = train[train$THERAPY == 0, ], 
                           mtry = all_tune$mtry[j], nodesize = all_tune$nodesize[j])
    model1 <- randomForest(Health ~ . - BEST, data = train[train$THERAPY == 1, ], 
                           mtry = all_tune$mtry[j], nodesize = all_tune$nodesize[j])

    # now use model0 and model1 to predict health outcomes on each of the
    # patients in the testing set
    test0 <- predict(model0, test)
    test1 <- predict(model1, test)

    # for each patient in the testing set, recommend whichever treatment has the
    # greater score in the model predictions
    suggest <- (test1 > test0)
    # compare the suggestions to the BEST (i.e., doctor-recommended) treatment
    # and record the mean error
    error_vals[j, k] <- mean(suggest != test$BEST)
  }
}

# summarize the error for each combination of nodesize and mtry evaluated
cbind(all_tune, "Mean Error" = rowMeans(error_vals))

# Interpretation

Which values of `nodesize` and `mtry` seem best? What might that say about the underlying truth in the data set?

# Single-Tree Model

As a second step, we will construct a single-tree model (CART) to represent the choice of best treatment.

Read the code and comments in the cell below. Run the cell, which will take less than a second to complete.

In [0]:
# build a new model0 and model1 using the optimal mtry and nodesize we found above
# this time, there's no split into training and tests sets; all patients for each
# therapy are included, but we still do not use the BEST column
model0 <- randomForest(Health ~ . - BEST, data =  Sepsis[Sepsis$THERAPY == 0, ], mtry = 12, nodesize = 10)
model1 <- randomForest(Health ~ . - BEST, data =  Sepsis[Sepsis$THERAPY == 1, ], mtry = 12, nodesize = 10)

# calculate the predicted health for all patients according to model0 and model1
pred0 <- predict(model0, Sepsis)
pred1 <- predict(model1, Sepsis)

# determine the suggested treatment according to the predicted health
Sepsis$FitBest <- (pred1 > pred0)

# fit a decision tree to predict FitBest, excluding BEST, Health, and THERAPY
rpart.fit <- rpart(as.factor(FitBest) ~ . - BEST - Health - THERAPY, data = Sepsis)

# in the coming cells, we will prune the tree
# start by plotting the cross-validation relative error at different tree sizes
plotcp(rpart.fit)

Run the cell below, which will take less than a second, to view the plot data in tabular form.

In [0]:
rpart.fit$cptable

When pruning, we want to choose a value of `cp` that balances the tree complexity against the error performance. According to the [plotcp documentation](https://www.rdocumentation.org/packages/rpart/versions/4.1-15/topics/plotcp), "A good choice of `cp` for pruning
is often the leftmost value for which the mean lies below the horizontal line." What is the intuition behind that guidance? What value of `cp` is suggested by the plot and table above? 

Run the cell below, which will take less than a second, to build and view a pruned tree.

In [0]:
rpart.fit <- rpart(as.factor(FitBest) ~ . - BEST - Health - THERAPY, data = Sepsis, cp = 0.023)

plot(rpart.fit)
text(rpart.fit, use.n = TRUE)